##### Import Package

In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import konlpy.tag 
import pandas as pd
import scipy as sp
import operator
import numpy as np

##### Get Article & Comment

In [2]:
def shuffle_df(df):
    return df.iloc[np.random.permutation(len(df))].reset_index(drop=True) 

def get_datas(date, comment=1):
    article_df = pd.read_csv("./news/" + date + ".csv")
    article_df = shuffle_df(article_df)
    comment_df = pd.read_csv("./comment/comment"+str(comment)+".csv")
    return article_df, comment_df

#####  Morpheme Analytics

In [3]:
def make_document_morpheme(texts):
    """make documents list after morphological"""
    
    documents = []
    
    komoran = konlpy.tag.Komoran()
    
    for text in texts:
        obj = line2obj(komoran.pos(text, flatten=False))
        documents.append(obj)
        
    return documents

def line2obj(lines):
    """ make keywords dictionary include only (NNP, NNG)"""
    
    obj = {}
    
    for line in lines:
        for keyword in line:
            if len(keyword[0]) > 1 and ( keyword[1] == "NNP" or keyword[1] == "NNG" ) :
                key = keyword[0]
                if key in obj:
                    obj[key] += 1
                else:
                    obj[key] = 1
                    
    return obj

In [4]:
def gen_text_documents(documents):
    
    text_documents = []
    
    for document in documents:
        
        text_words = []
        
        for word, count in document.items():
            text_words.extend([word] * count)
            
        text_document = " ".join(text_words)
        text_documents.append(text_document)
        
    return text_documents        

In [5]:
def calc_euc_dist(v1, v2):
    delta = v1 - v2
    dist = sp.linalg.norm(delta.toarray())
    return dist

def get_euc_dists(vectorized, num_docs):
    
    dists = []
    
    for idx in range(num_docs-1):
        dist = calc_euc_dist(vectorized.getrow(num_docs-1), vectorized.getrow(idx))
        dists.append((num_docs-1, idx, dist))
        
    return dists

def sort_dists(dists):
    result_list = []
    for i, j, dist in sorted(dists, key=operator.itemgetter(2)):
        result_list.append((i, j, dist))
    return result_list

##### Text Preprocessing

In [31]:
# get data
article_df, comment_df = get_datas("2016-07-07", 2)
len(article_df), len(comment_df)

(1873, 110)

In [32]:
# make documents
comment_str = comment_df["content"][:].str.cat(sep=' ')
article_series = article_df["content"][:100]
article_series = article_series.append(pd.Series([comment_str])).reset_index(drop=True)
%time article_documents = make_document_morpheme(article_series)

CPU times: user 10.7 s, sys: 261 ms, total: 11 s
Wall time: 5.31 s


In [33]:
# make gen documents
gen_articles = gen_text_documents(article_documents)

##### Vectorizer

In [34]:
# vectorizer = CountVectorizer()
vectorizer = TfidfVectorizer()
# vectorizer = HashingVectorizer()

vectorized = vectorizer.fit_transform(gen_articles)
num_docs, num_features =  vectorized.shape
feature_names = vectorizer.get_feature_names()

print("the number of input documents : {}".format(num_docs))
print("the number of words : {}".format(num_features))
result_df = pd.DataFrame(vectorized.toarray(), columns=feature_names)

the number of input documents : 101
the number of words : 6007


##### Measurement Distance between comment and document

In [35]:
dists = get_euc_dists(vectorized, num_docs)
result = sort_dists(dists)
result[:10]

[(100, 58, 1.3118112301585028),
 (100, 9, 1.3236153224507836),
 (100, 23, 1.3305864643786132),
 (100, 62, 1.3315614933374107),
 (100, 50, 1.3349254704047226),
 (100, 7, 1.3465324207978364),
 (100, 79, 1.3472167776139905),
 (100, 37, 1.3527655933910587),
 (100, 24, 1.3546201061158731),
 (100, 22, 1.3559338154407272)]

In [36]:
# comment
print(article_series[comment[0]][0:100])

[ 하늘은 어째서 박정희를 다시 불러냈는가!!]

아래 댓글에서 삿갓은 빅정희를 다시 불러낸 것은 하늘이라고 주장했다.
그러면 하늘은 어째서 박정희를 다시 불러냈는가.

이순신이나


##### Recomend Article

In [38]:
comment, rank_idx, distance = zip(*result)
idx = rank_idx[0]
print(idx)
print(article_series[idx].split("▶")[0])

58
국정원 대선개입 논란에 남북정상회담 회의록 유출은 이미 유죄  [CBS노컷뉴스 김효은 기자]박근혜 대통령(사진=청와대 제공)"컴퓨터를 빼앗기면 국가정보원 심리전단 소속 직원들의 인터넷 게시글 등 대선 개입 활동 내용이 수사기관·언론 등에 공개될 수 있다는 점에 대한 두려움을 느껴 스스로 밖으로 나가지 않은 것이다"법원이 지난 6일 국정원 여직원 감금 혐의로 기소된 민주통합당(현 더불어민주당) 이종걸 의원과 문병호·강기정·김현 전 의원에게 무죄를 선고하면서 밝힌 내용이다. 18대 대통령 선거를 일주일 앞둔 2012년 12월 11일 이 의원 등이 들이닥친 서울 역삼동 오피스텔 안에서는 국정원 심리전단 소속 여직원인 김모씨가 인터넷 게시판 등에서 여권에 유리한 정치 댓글을 달고 있었다.김씨는 노트북과 컴퓨터를 제출하라는 민주통합당 관계자와 경찰의 거듭된 요구에도 문을 걸어잠근 채 밖으로 나오지 않았다. 법원은 김씨가 나오지 않은 이유를 대선 개입이 알려질까봐 두려워했기 때문이라고 봤다.지난 2012년 12월 13일 민주통합당 문재인 후보 측에 악성댓글 유포 혐의를 받고 있는 국정원 여직원 김 모 씨가 역삼동 자신의 오피스텔에서 경찰의 증거자료를 압수를 지켜보고 있다. (사진=황진환 기자)국민이 직접 대통령을 뽑는 직선제 국가에서 국정원의 대선 개입은 명백한 관권선거다. 정보기관의 비호를 받은 정권은 정치적 정당성에 치명타를 입을 수밖에 없다.이 때문에 박근혜 대통령은 임기 초반부터 관권선거 시비에 시달려야만 했다. 원세훈 전 국정원장의 대선 개입 사건과 남북정상회담 대화록 유출 사건, 국정원 여직원 감금 사건의 당사자들이 줄줄이 법정에 섰다.가장 먼저 결론이 난 대화록 유출 사건을 제외하면 나머지 두 사건은 아직 사법부의 최종 판단을 기다리고 있다. 19대 대선이 치러지는 내년 12월까지 1년 6개월의 시간이 남아있다는 점을 감안할 때 박 대통령은 임기 말까지 관권선거 의혹에서 완전히 벗어나지는 못할 듯하다. 국정원 직원들에게 정치 댓글 작성을 지시한 혐의로 